In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_core.output_parsers import StrOutputParser

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(
    model="gpt-4o-mini"
    )

In [6]:
from langchain_core.prompts import HumanMessagePromptTemplate,AIMessagePromptTemplate

example_prompt=(
    HumanMessagePromptTemplate.from_template("{description}")
    +AIMessagePromptTemplate.from_template("{classification}")
)

In [7]:
examples=[
    {
        "description":"食物偏甜",
        "classification":"南部人"
    },
    {
        "description":"食物偏鹹",
        "classification":"北部人"
    },
    {
        "description":"滷肉飯",
        "classification":"北部人"
    },
    {
        "description":"肉燥飯",
        "classification":"南部人"
    },
    {
        "description":"搭乘大眾運輸，不怕走路",
        "classification":"北部人"
    },
    {
        "description":"騎摩托車。不待轉",
        "classification":"南部人"
    },
]

In [8]:
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate

few_shot_prompt=FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)

In [9]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

from langchain_core.prompts.chat import ChatPromptTemplate

final_prompt=ChatPromptTemplate.from_messages(
    [
        ("system","請根據以下描述，判斷是哪一種人"),
        few_shot_prompt,
        ("human","{input}")
    ]
)


chain=final_prompt | llm |parser

In [10]:
user_input="甜味"
response=chain.invoke({"input":user_input})
print("描述:",user_input)
print("分類:",response)

描述: 甜味
分類: 喜歡甜食的人。


In [11]:
from langchain.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

embedding_funtion=OpenAIEmbeddings(
     model="text-embedding-3-small"
)

In [12]:
example_selector=SemanticSimilarityExampleSelector.from_examples(
    examples,embedding_funtion,Chroma,k=1
)

In [13]:
question="騎車經過十字路口會直接左轉"


select_examples=example_selector.select_examples(
    {"description":question}
)

print(question)
print("最相似的例子是 : ")

for example in select_examples:
    for key,value in example.items():
        print(f"{key}:{value}")
    print("\n")
    

騎車經過十字路口會直接左轉
最相似的例子是 : 
classification:南部人
description:騎摩托車。不待轉




In [15]:
few_shot_prompt_v2=FewShotChatMessagePromptTemplate(
    # examples=examples,
    example_prompt=example_prompt,
    example_selector=example_selector
)

print(few_shot_prompt_v2.invoke({"description":"喜歡吃甜甜"}))

messages=[HumanMessage(content='食物偏甜', additional_kwargs={}, response_metadata={}), AIMessage(content='南部人', additional_kwargs={}, response_metadata={})]


In [16]:
final_prompt_v2=ChatPromptTemplate.from_messages(
    [
        ("system","請根據以下精選描述，判斷是北部人還是南部人"),
        few_shot_prompt,
        ("human","{input}")
    ]
)


chain_v2=final_prompt | llm |parser

In [19]:
user_input="甜味"
response=chain_v2.invoke({"input":user_input})
print("描述:",user_input)
print("分類:",response)

描述: 甜味
分類: 喜歡甜食的人。


In [18]:
user_input="醬油有甜味"
response=chain_v2.invoke({"input":user_input})
print("描述:",user_input)
print("分類:",response)

描述: 醬油有甜味
分類: 南部人


In [25]:
final_prompt_v2=ChatPromptTemplate.from_messages(
    [
        ("system","""
                    請根據以下精選描述，判斷是北部人還是南部人
                    只回答北部人或南部人即可
                """
         
         ),
        few_shot_prompt,
        ("human","{input}")
    ]
)


chain_v2=final_prompt | llm |parser

In [28]:
user_input="有甜味"
response=chain_v2.invoke({"input":user_input})
print("描述:",user_input)
print("分類:",response)

描述: 有甜味
分類: 可以是指南部人，因為南部的食物通常偏甜。
